In [ ]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from joblib import delayed
from IPython.display import clear_output
import sys
sys.path.append(os.path.join(os.getcwd(), ".."))

from attractor_creation.attractor_generator import Attractor_Generator
from shared_utils.utils_parallel import ParallelExecutor

CWD = os.getcwd()

In [ ]:
high_noise = {
    "chua": 0.01,
    "duffing": 1.5,
    "lorenz": 0.5,
    "lotka_volterra": 1.0,
    "nose_hover": 1.0,
    "rikitake": 0.5,
    "rossler": 1.,
    "random_noise3d": 1.0,
    "becker": 0.5
}

In [ ]:
def generate_series(name, start_idx, save_path, i):
    attractors = Attractor_Generator()
    # w = np.random.uniform(low=1e-12, high=0.01)
    w = np.random.uniform(low=0.01, high=high_noise[name])
    attractors.compute_attractor(
        attractor=name, x0=0.6 + w, y0=0.2 + w, z0=0.1 + w, dt=0.01, num_steps=3500)
    attractors.coordinates = attractors.coordinates[start_idx:,:]
    attractors.t = attractors.t[start_idx:]
    # results = attractors.get_data()

    attractors.generate_data_file(path=save_path, filename=name + "__" + str(i))

In [ ]:
nb_simulation = 100

attractors_concerned = ["lorenz","rossler"]
for attract in attractors_concerned:
    save_path = os.path.join(CWD, "..","..","data", f"{attract}_attractors")
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    aprun = ParallelExecutor(n_jobs=-1)
    aprun(total=nb_simulation)(
                delayed(generate_series)(attract, 300, save_path,i) for i in range(nb_simulation)
            )
            
clear_output()